# Volume of Histogram 
Imagine a histogram (bar graph). Design an algorithm to compute the volume of water it could hold if someone poured water across the top. You can assume each histogram bar has width 1.

### Solution 1: Brute Force
In this solution, you need to find the max value of the histogram (or a max if there's more than one), and imagine pouring water at that point. In this case, water would flow left and right from this point, filling the lower bars (or empty slots) with water. This can be done with the following steps:

1. Find the/a max in the array.
2. Find the/a max in the left subarray (start to max - 1)
3. Calculate the water between these two peaks
4. Move onto the leftmost max and repeat until you reach the beginning of the array.
3. Do the same for the right subarray (max + 1 to end), repeating until the end.

This is a fairly straightforward way to approach this problem, and it's nice that this solution takes O(1) space. However, in the worst case (a sorted list asc/desc), this algorithm performs on the order of O(n^2). 

In [2]:
import math

def fillWithWater(a):
    leftPeak = getPeakIndex(a, 0, len(a))
    rightPeak = leftPeak
    water = 0
    # go left
    while leftPeak > 0:
        nextPeak = getPeakIndex(a, 0, leftPeak)
        water += howMuchWater(a, nextPeak, leftPeak)
        leftPeak = nextPeak
    # go right
    while rightPeak < (len(a) - 1):
        nextPeak = getPeakIndex(a, rightPeak + 1, len(a))
        water += howMuchWater(a, rightPeak, nextPeak)
        rightPeak = nextPeak
    return water

def howMuchWater(a, start, end):
    waterLevel = min(a[start], a[end])
    water = 0
    for i in range(start + 1, end):
        water += waterLevel - a[i]
    return water

def getPeakIndex(a, start, end):
    peakIndex = start
    for i in range(start, end):
        if a[i] > a[peakIndex]:
            peakIndex = i
    return peakIndex

### Solution 2: Precomputing an array of maxes to the right.
###### *Disclaimer: this solution is crazy.*

Realize that, in the previous solution, the problem was with `getPeakIndex`. In a sorted list, the next peak is always right next to the current peak, so not only does the histogram not even hold water, `getPeakIndex` turns the algorithm into O(n^2).

So how do we fix that? The key insight here is that you can compute an array of the maxes on the left/right by rolling forward/backward through the array, respectively. So now the steps become:

1. Compute the array of maxes from the right.
2. Compute the array of maxes from the left.
3. For each bar in the histogram, subtract its height from the smallest peak on either side, and add that to the sum.

##### An Example
Imagine you have the array `[3, 1, 2, 4, 1, 1, 2]`. 

* If you start from the right and move left, you can always be sure to know the next highest element to your right. So if you compute this array, you would get `[4, 4, 4, 2, 2, 2, 2]`. 
* Similarly, the maxes from the left would be `[3, 3, 3, 4, 4, 4, 4]`, but you don't actually have to compute this array, you can just keep track of the max when going through the array in the next step.
* As you iterate through the histogram, you compare the peaks on either side of the current element. Whichever one is lower dictates the water level, so simply subtract the current bar height from the smaller of its peaks. At the end of the pass, you're fucking done.

In [3]:
def fillWithWaterOptimized(a):
    rightMaxes = getMaxesFromRight(a)
    water = 0
    curLeftMax = a[0]
    for index, height in enumerate(a):
        curLeftMax = max(height, curLeftMax)
        water += min(curLeftMax, rightMaxes[index]) - height
    return water

def getMaxesFromRight(a):
    end = len(a) - 1
    maxArray = [None] * len(a)
    maxHeight = a[end]
    for i, height in enumerate(reversed(a)):
        maxHeight = max(height, maxHeight)
        maxArray[end - i] = maxHeight
    return maxArray

### Here are some tests.
In case you don't believe me...

In [4]:
tests = []
tests.append([3, 1, 2, 4, 3, 2, 1])
tests.append([0, 0, 0])
tests.append([7])
tests.append([0, 7, 0])
tests.append([7, 7, 7])
tests.append([2, 1, 2])
tests.append([2, 0, 3, 0, 1])
tests.append([1, 2, 3, 4])
tests.append([4, 3, 2, 1])
tests.append([1, 3, 2, 1, 3, 3, 1, 2, 3, 1, 2, 0, 0, 2])

for histogram in tests:
    print("*****the normal answer for", histogram, "is:", fillWithWater(histogram))    
    print("*****the better answer for", histogram, "is:", fillWithWaterOptimized(histogram))

*****the normal answer for [3, 1, 2, 4, 3, 2, 1] is: 3
*****the better answer for [3, 1, 2, 4, 3, 2, 1] is: 3
*****the normal answer for [0, 0, 0] is: 0
*****the better answer for [0, 0, 0] is: 0
*****the normal answer for [7] is: 0
*****the better answer for [7] is: 0
*****the normal answer for [0, 7, 0] is: 0
*****the better answer for [0, 7, 0] is: 0
*****the normal answer for [7, 7, 7] is: 0
*****the better answer for [7, 7, 7] is: 0
*****the normal answer for [2, 1, 2] is: 1
*****the better answer for [2, 1, 2] is: 1
*****the normal answer for [2, 0, 3, 0, 1] is: 3
*****the better answer for [2, 0, 3, 0, 1] is: 3
*****the normal answer for [1, 2, 3, 4] is: 0
*****the better answer for [1, 2, 3, 4] is: 0
*****the normal answer for [4, 3, 2, 1] is: 0
*****the better answer for [4, 3, 2, 1] is: 0
*****the normal answer for [1, 3, 2, 1, 3, 3, 1, 2, 3, 1, 2, 0, 0, 2] is: 11
*****the better answer for [1, 3, 2, 1, 3, 3, 1, 2, 3, 1, 2, 0, 0, 2] is: 11
